In [0]:
%run
/Workspace/Repos/h20240186@pilani.bits-pilani.ac.in/spark-olist-pipeline/src/utils/config

In [0]:
from pyspark.sql.functions import date_format

def transform_bronze(df, partition_col=None, repartition_rows=None):
    if partition_col:
        df = df.withColumn("order_month", date_format(df[partition_col], "yyyy-MM"))
        if repartition_rows:
            df = df.repartition(repartition_rows, "order_month")  # Only if explicitly requested
    return df


def ingest_to_bronze(spark, cfg):
    df = (spark.read.format("csv")
                  .option("header", True)
                  .option("inferSchema", True)
                  .load(cfg["input_path"]))
    df = transform_bronze(df, cfg.get("partition_col"))
    writer = (df.write.format("delta")
                  .option("maxRecordsPerFile", 1_000_000)
                  .mode("overwrite"))
    if cfg.get("partition_col"):
        writer = writer.partitionBy("order_month")
    writer.save(cfg["output_path"])
    return df


In [0]:

for table, cfg in bronze_sources.items():
    ingest_to_bronze(spark, cfg)

In [0]:
#inspecting schema for transformations

def print_schema(link):
    df = spark.read.format("delta").load(link)
    df.printSchema()



for key,value in bronze_sources.items():
    print_schema(value["output_path"])